In [51]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [64]:
df = pd.read_csv('capture20110815-2.pcap.netflow.labeled.csv')
# print(df.head())

#preprocessing
df = df[df['Prot']!='IPV6-ICMP']

ips = df['Src_IP_Addr:Port'].str.split(':')
df['Src_IP_Addr'] = ips.str[0]
df['Src_IP_Port'] = ips.str[-1]
ips = df['Dst_IP_Addr:Port'].str.split(':')
df['Dst_IP_Addr'] = ips.str[0]
df['Dst_IP_Port'] = ips.str[-1]

# df.dtypes
# ips = df['Src_IP_Addr:Port'].str.split(':')
# print(pd.sort(ips.str[0:-1]))
# df['Src_IP_Addr'] = ':'.join(ips.str[0:-1])
# df['Src_IP_Port'] = ips.str[-1]
# ips = df['Dst_IP_Addr:Port'].str.split(':')
# df['Dst_IP_Addr'] = ':'.join(ips[0:-1])
# df['Dst_IP_Port'] = ips.str[-1]
df.head()

C:\Users\Anna\AppData\Local\Programs\Python\Python36-32\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Date_flow_start,time,Durat,Prot,Src_IP_Addr:Port,Dst_IP_Addr:Port,Flags,Tos,Packets,Bytes,Flows,Label,Labels,Src_IP_Addr,Src_IP_Port,Dst_IP_Addr,Dst_IP_Port
0,2011-08-15,16:43:21.xxx,0.235,TCP,147.32.84.59:64131,164.8.32.159:54992,PA_,0,3,211,1,Background,NaN,147.32.84.59,64131,164.8.32.159,54992
1,2011-08-15,16:43:21.xxx,0.444,TCP,147.32.87.252:49218,147.32.80.13:3128,FPA_,0,300,19566,1,LEGITIMATE,NaN,147.32.87.252,49218,147.32.80.13,3128
2,2011-08-15,16:43:21.xxx,4.993,TCP,41.143.58.184:1328,147.32.86.179:58675,PA_,0,269,29373,1,Background,NaN,41.143.58.184,1328,147.32.86.179,58675
3,2011-08-15,16:43:21.xxx,0.187,TCP,147.32.86.155:51273,204.154.94.81:80,FA_,0,2,132,1,Background,NaN,147.32.86.155,51273,204.154.94.81,80
4,2011-08-15,16:43:21.xxx,0.442,TCP,147.32.80.13:3128,147.32.87.252:49218,FPA_,0,579,728343,1,LEGITIMATE,NaN,147.32.80.13,3128,147.32.87.252,49218


In [48]:
# df['full_date'] = pd.to_datetime(df['Date_flow_start'] + ' ' + df['time'])

# df.groupby(['Src_IP_Addr']).head()
df.groupby(['Src_IP_Addr'])['Src_IP_Addr'].count()

Src_IP_Addr
0                216
0.0.0.0          136
1.1.224.18        12
1.114.187.73       2
1.144.165.119      3
1.160.235.236      1
1.161.139.171      1
1.168.194.160      1
1.168.203.10       1
1.169.130.121      1
1.169.155.97       1
1.169.173.94       5
1.169.217.36       1
1.170.132.94       1
1.170.140.94       1
1.171.107.97       2
1.173.166.53       1
1.174.130.115      1
1.174.134.176      1
1.175.128.119      1
1.175.130.54       2
1.195.6.36         1
1.202.19.240       1
1.23.180.225       1
1.23.215.236       1
1.36.69.113        1
1.47.52.31         1
1.52.188.130       1
1.53.40.172        1
1.53.52.171        2
                ... 
99.71.173.50       1
99.71.242.175      1
99.72.221.137      4
99.73.113.11       1
99.73.247.51       1
99.74.39.166       1
99.75.167.178      1
99.8.15.206        1
99.83.7.149        2
99.88.236.127      3
99.89.115.5        1
99.89.90.101       1
99.9.112.124       1
99.9.173.237       6
99.9.194.217       1
99.9.94.61         1
9